In [ ]:
import numpy as np
import pandas as pd
import yaml
import matplotlib.pyplot as plt

from pathlib import Path

In [ ]:
paths = list(Path.home().glob("experiments/experiments/C4P4/ilp.timelimit=100.threads=1.constraints=sparse/bio/*"))

In [ ]:
docs = []
for path in paths:
    with path.open() as file:
        docs += list(yaml.safe_load_all(file))

In [ ]:
df = pd.DataFrame(docs)
df = pd.concat([df.drop(["instance", "config"], axis=1)] + [df[x].apply(pd.Series) for x in ["instance"]], axis=1)
df = df.astype({k: "category" for k in ["commit_hash", "forbidden_subgraphs"]})
df["name"] = df["name"].str.replace(".*/", "")
df["solved"] = df["solution_cost"] >= 0

df["solution_length"] = df["solutions"].apply(lambda x: len(x[0]["edits"]) if len(x) > 0 else -1)

df

In [ ]:
g = df[df["solution_cost"] > 0]
g = g.sort_values("solution_cost")

fig, ax = plt.subplots()

ax.grid(True)
ax.set_yscale("log")
#ax.set_xscale("log")

x = range(len(g))
y = g.solution_cost
#y = g.solution_length

ax.scatter(x, y)

ax.set_xlabel("Graphs")
ax.set_ylabel("$k$")

ax.set_ylim((1, y.max()))
plt.show()

In [ ]:
(df["solution_cost"] == 0).sum(), (df["solution_cost"] == -1).sum(), (df["solution_cost"] > 0).sum()

In [ ]:
df[df["solution_cost"] == 0][df["solution_length"] == 1]

In [ ]:
g = df[(df["solution_length"] >= 10) | (~df["solved"])].copy()
g.loc[~g["solved"], "time"] = g["time"].max() * 10

g = g.sort_values("time")

fig, ax = plt.subplots()

ax.set_xscale("log")

x = g["time"] / 10**9

ax.plot(x, range(len(g)))


ax.set_xlim((None, 100))
plt.show()

In [ ]:
g = df[df["solved"]]

fig, ax = plt.subplots()

ax.set_yscale("log")

ax.scatter(g["solution_length"], g["time"])

ax.set_ylim((g["time"].min(), g["time"].max()))

plt.show()